In [1]:
# !pip install transformers accelerate
# !pip install transformers[sentencepiece]
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
# !pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
label2id = {'positive':0,'negative':1}

In [3]:
train = pd.read_csv(r'dataset I/train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
train = train[['label','text']].reset_index(drop=True)

test = pd.read_csv(r'dataset I/test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

test = test[['label','text']].reset_index(drop=True)

In [4]:
# df

In [5]:
# train = train_II
train = train.replace({'label':label2id})
# test = test_II
test = test.replace({'label':label2id})
# train, test = train_test_split(df,test_size=0.7,random_state=1)

In [6]:
train['label'].value_counts()

label
1    4443
0    2635
Name: count, dtype: int64

In [7]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test.iterrows():
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)

# for index, row in test.iterrows():
#     input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
#     dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "XSS \ucde8\uc57d\uc810 \uc218\uc815",
  "text_b": "",
  "tgt_text": null
}



In [8]:
len(train)

7078

In [9]:
len(test)

3034

In [10]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "google/flan-t5-base")


In [11]:
tuned_params = filter(lambda p: p.requires_grad, plm.parameters())
num_tuned_params = sum([p.numel() for p in tuned_params])

print(f"Number of Tuned Parameters: {num_tuned_params}")

Number of Tuned Parameters: 247577856


In [12]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' XSS 취약점 수정', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [13]:
# WrapperClass.save_pretrained(nb_fname)

In [14]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 3, 4, 4256, 3, 2, 3, 2, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁', 'X', 'SS', '▁', '<unk>', '▁', '<unk>', '▁is', '▁', 'a', '<extra_id_0>', '▁commit', '.', '</s>', '<pad>', '<p

In [15]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [16]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 7078it [00:15, 450.33it/s]


In [17]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
myverbalizer = ManualVerbalizer(tokenizer, num_classes=2,
                        label_words=[["positive"], ["negative"]])
# myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=3).from_file("knowledgeable_verbalizer-II.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


Parameter containing:
tensor([[[1465]],

        [[2841]]])
tensor([[-0.3223, -1.2890],
        [-0.4326, -1.0465]])


In [18]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [19]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
import ipynbname
nb_fname = ipynbname.name()
print(nb_fname)

M-Flan-T5-PKG_I


In [21]:
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 3034it [00:06, 443.38it/s]


In [22]:
# Evaluate
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0

for epoch in tqdm(range(10)):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        # torch.save(prompt_model.state_dict(), nb_fname +'.pth')
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, average loss: 1.134253978729248
Epoch 0, average loss: 0.5382757996227227
Epoch 0, average loss: 0.5060563127851427
Epoch 0, average loss: 0.46661763456927624
Epoch 0, average loss: 0.4398134661773544
Epoch 0, average loss: 0.41701447463546143
Epoch 0, average loss: 0.40717857504306837
Epoch 0, average loss: 0.39958577668639245
Epoch 0, average loss: 0.38438523856300555
Epoch 0, average loss: 0.37711018273924907
Epoch 0, average loss: 0.3708675158862582
Epoch 0, average loss: 0.36152958063647056
Epoch 0, average loss: 0.35589091022136293
Epoch 0, average loss: 0.34751534446918003
Epoch 0, average loss: 0.3449057764842038
Epoch 0, average loss: 0.34275184891350347
Epoch 0, average loss: 0.33704014893733986
Epoch 0, average loss: 0.3329011562291848
Epoch 0, validation loss: 0.2903853853700047


 10%|█         | 1/10 [07:49<1:10:23, 469.27s/it]

Epoch 1, average loss: 0.17865132354199886
Epoch 1, average loss: 0.16236521235174117
Epoch 1, average loss: 0.16928906736913885
Epoch 1, average loss: 0.1747350266842412
Epoch 1, average loss: 0.1658964615216159
Epoch 1, average loss: 0.15876327391850217
Epoch 1, average loss: 0.161541785958008
Epoch 1, average loss: 0.16843283060487715
Epoch 1, average loss: 0.1702123085320509
Epoch 1, average loss: 0.16610585362043856
Epoch 1, average loss: 0.17140886793761215
Epoch 1, average loss: 0.1693103797578493
Epoch 1, average loss: 0.16987437555965845
Epoch 1, average loss: 0.17045229660820901
Epoch 1, average loss: 0.17017372734781164
Epoch 1, average loss: 0.17059834223474607
Epoch 1, average loss: 0.17156580566246857
Epoch 1, average loss: 0.17151571637563356
Epoch 1, validation loss: 0.2996951587225095


 20%|██        | 2/10 [15:55<1:03:51, 478.96s/it]

Epoch 2, average loss: 0.0186802432872355
Epoch 2, average loss: 0.09875604192847294
Epoch 2, average loss: 0.0747717162608607
Epoch 2, average loss: 0.07973629567060551
Epoch 2, average loss: 0.07773751962412828
Epoch 2, average loss: 0.07878992069025667
Epoch 2, average loss: 0.07814919796583489
Epoch 2, average loss: 0.0883542775668256
Epoch 2, average loss: 0.09093153586756689
Epoch 2, average loss: 0.09049164285628036
Epoch 2, average loss: 0.0926309418720445
Epoch 2, average loss: 0.0940508768234974
Epoch 2, average loss: 0.09400350185811679
Epoch 2, average loss: 0.09701982399982893
Epoch 2, average loss: 0.09608259030273375
Epoch 2, average loss: 0.09774100115285057
Epoch 2, average loss: 0.09779965619675883
Epoch 2, average loss: 0.09964555515139539
Epoch 2, validation loss: 0.4037623214767546


 30%|███       | 3/10 [24:02<56:20, 482.96s/it]  

Epoch 3, average loss: 0.20962756127119064
Epoch 3, average loss: 0.06700294960847161
Epoch 3, average loss: 0.06236468851977298
Epoch 3, average loss: 0.05503001438195714
Epoch 3, average loss: 0.05594880443130078
Epoch 3, average loss: 0.06405583204876174
Epoch 3, average loss: 0.06572949220327907
Epoch 3, average loss: 0.0655639120156225
Epoch 3, average loss: 0.06550641796493914
Epoch 3, average loss: 0.06832066666014375
Epoch 3, average loss: 0.06923634356329415
Epoch 3, average loss: 0.06987300552100104
Epoch 3, average loss: 0.07109874743954668
Epoch 3, average loss: 0.07027047559141869
Epoch 3, average loss: 0.06798853859114652
Epoch 3, average loss: 0.06743553946981422
Epoch 3, average loss: 0.06704946681054948
Epoch 3, average loss: 0.06950862012662273
Epoch 3, validation loss: 0.43536343044662607
Early stopping at epoch 3.


 30%|███       | 3/10 [32:08<1:14:59, 642.80s/it]


In [23]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
# 这里是数字所以要转字符
class_names = list(test['label'].unique())  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, digits=4)

print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.8159    0.9062    0.8587      1130
           1     0.9404    0.8787    0.9085      1904

    accuracy                         0.8889      3034
   macro avg     0.8782    0.8924    0.8836      3034
weighted avg     0.8941    0.8889    0.8900      3034



In [24]:
# # store the checkpoints
# prompt_model.plm.save_pretrained(nb_fname)
# model_config.save_pretrained(nb_fname)
# myverbalizer.tokenizer.save_pretrained(nb_fname)

In [25]:
# import os
# os.system('shutdown')

In [26]:
# train.to_csv('dataset II/train.csv',encoding='utf_8_sig')
# test.to_csv('dataset II/test.csv', encoding='utf_8_sig')

In [27]:
# import os
# os.system('shutdown')

In [28]:
tuned_params = filter(lambda p: p.requires_grad, prompt_model.parameters())
num_tuned_params = sum([p.numel() for p in tuned_params])

print(f"Number of Tuned Parameters: {num_tuned_params}")

Number of Tuned Parameters: 247577856


In [29]:
!nvidia-smi

Tue Aug 15 03:13:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P40                      On  | 00000000:89:00.0 Off |                  Off |
| N/A   54C    P0              62W / 250W |   6194MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--